<a href="https://colab.research.google.com/github/sophiepavia/RESDIE/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-task Learning with GCN for Graph and Node level predictions about Neighborhood Change

## Libraries Used

In [52]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
# !pip install torch torchvision --quiet
# !pip install torch-geometric --quiet

In [3]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F

In [4]:
# Install datacommons_pandas
# !pip install datacommons_pandas --upgrade --quiet
# Import Data Commons
import datacommons_pandas as dc

# Import other required libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import geopandas as gpd

import json
import pickle

## Multi-task GCN Model

In [5]:
class MultitaskGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes_node, num_classes_graph):
        super(MultitaskGCN, self).__init__()
        # Define the first GCN layer
        self.conv1 = GCNConv(num_node_features, 16)
        # Define the second GCN layer
        self.conv2 = GCNConv(16, 16)

        # MLP for node-level prediction
        self.mlp_node = torch.nn.Sequential(
            torch.nn.Linear(16, 16),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(16, num_classes_node)
        )

        # MLP for graph-level prediction
        self.mlp_graph = torch.nn.Sequential(
            torch.nn.Linear(16, 16),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(16, num_classes_graph)
        )

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Shared feature extraction
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x)
        # x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.leaky_relu(x)

        # Global mean pooling for graph-level features
        x_pool = global_mean_pool(x, batch)

        # Node-level prediction
        out_node = self.mlp_node(x)

        # Graph-level prediction
        out_graph = self.mlp_graph(x_pool)

        return out_node, out_graph
        # return F.log_softmax(out_node, dim=1), F.log_softmax(out_graph, dim=1)

## Training

In [46]:
def train(model, optimizer, loader, loss_fn_node, loss_fn_graph):
  epochs = 500
  for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_batches = len(loader)

    for data in loader:  # Iterate over each batch of data
      optimizer.zero_grad()

      out_node, out_graph = model(data)
      # print(f'Max node output: {torch.max(out_node).item()}, Max graph output: {torch.max(out_graph).item()}')

      # Node-level loss
      loss_node = loss_fn_node(out_node, data.y)

      # Graph-level loss
      loss_graph = loss_fn_graph(out_graph, data.y_graph)

      # Combine losses; you might want to weight these differently
      loss = loss_node + loss_graph
      loss.backward()
      # print(f'Gradient norm (node): {torch.norm(next(model.parameters()).grad)}')
      optimizer.step()

      total_loss += loss.item()

    average_loss = total_loss/total_batches
    print(f'Epoch {epoch + 1}, Average Loss: {average_loss:.4f}')


## Test

In [50]:
def evaluate(model, loader):
    model.eval()
    true_labels_node = []
    predictions_node = []
    true_labels_graph = []
    predictions_graph = []

    with torch.no_grad():
        for data in loader:  # Process each batch
            out_node, out_graph = model(data)
            _, pred_node = out_node.max(dim=1)
            _, pred_graph = out_graph.max(dim=1)

            # Append predictions and labels for later evaluation
            true_labels_node.extend(data.y.tolist())
            predictions_node.extend(pred_node.tolist())
            true_labels_graph.extend(data.y_graph.tolist())
            predictions_graph.extend(pred_graph.tolist())

    # Calculate metrics for node-level predictions
    node_precision = precision_score(true_labels_node, predictions_node, average='macro')
    node_recall = recall_score(true_labels_node, predictions_node, average='macro')
    node_f1 = f1_score(true_labels_node, predictions_node, average='macro')

    # Calculate metrics for graph-level predictions
    graph_precision = precision_score(true_labels_graph, predictions_graph, average='macro')
    graph_recall = recall_score(true_labels_graph, predictions_graph, average='macro')
    graph_f1 = f1_score(true_labels_graph, predictions_graph, average='macro')

    # Accuracy calculations
    node_acc = sum(1 for true, pred in zip(true_labels_node, predictions_node) if true == pred) / len(true_labels_node)
    graph_acc = sum(1 for true, pred in zip(true_labels_graph, predictions_graph) if true == pred) / len(true_labels_graph)

    return {
        'Node Accuracy': node_acc,
        'Node Precision': node_precision,
        'Node Recall': node_recall,
        'Node F1-Score': node_f1,
        'Graph Accuracy': graph_acc,
        'Graph Precision': graph_precision,
        'Graph Recall': graph_recall,
        'Graph F1-Score': graph_f1
    }


In [7]:
# def evaluate(model, loader):
#   model.eval()
#   total_correct_node = 0
#   total_nodes = 0
#   total_correct_graph = 0
#   total_graphs = 0

#   with torch.no_grad():
#     for data in loader:  # Process each batch
#       out_node, out_graph = model(data)
#       _, pred_node = out_node.max(dim=1)
#       correct_node = pred_node.eq(data.y).sum().item()
#       total_correct_node += correct_node
#       total_nodes += data.y.size(0)

#       _, pred_graph = out_graph.max(dim=1)
#       correct_graph = pred_graph.eq(data.y_graph).sum().item()
#       total_correct_graph += correct_graph
#       total_graphs += data.y_graph.size(0)

#   node_acc = total_correct_node / total_nodes
#   graph_acc = total_correct_graph / total_graphs

#   return node_acc, graph_acc

## Problem Set up: Census Data Exctraction

In [8]:
# Function to process data for a specific year
def process_data_for_year(year, data_dict):
    result = []
    for geo_id, metrics in data_dict.items():
        row = {'GeoID': geo_id}
        if metrics['Median_Income_Household'] and metrics['Median_HomeValue_HousingUnit_OccupiedHousingUnit_OwnerOccupied'] and metrics['Count_Person_EducationalAttainmentBachelorsDegreeOrHigher'] and metrics['Count_Person_WhiteAlone']:
          income_data = metrics['Median_Income_Household']['sourceSeries'][0]['val']
          if year in income_data:
              row[f'{year}_Median_Income'] = income_data[year]

          home_value_data = metrics['Median_HomeValue_HousingUnit_OccupiedHousingUnit_OwnerOccupied']['sourceSeries'][0]['val']
          if year in home_value_data:
              row[f'{year}_Median_Home_Value'] = home_value_data[year]

          education_data = metrics['Count_Person_EducationalAttainmentBachelorsDegreeOrHigher']['sourceSeries'][0]['val']
          if year in education_data:
              row[f'{year}_Bachelor_Degree'] = education_data[year]

          person_data = metrics['Count_Person']['sourceSeries'][0]['val']
          if year in person_data:
              row[f'{year}_Persons'] = person_data[year]

          race_data = metrics['Count_Person_WhiteAlone']['sourceSeries'][0]['val']
          if year in race_data:
              row[f'{year}_Count_White'] = race_data[year]

          result.append(row)
    return pd.DataFrame(result)

In [9]:
def get_features_census_tracts(df_2012,df_2019,income_percentile_40=None,home_value_percentile_40=None,education_threshold=None,home_threshold=None):
  df_2012["2012_Percentage_College"] = df_2012['2012_Bachelor_Degree']/df_2012['2012_Persons'] * 100
  df_2012["2012_Percentage_Non_White"] = (df_2012['2012_Persons'] - df_2012['2012_Count_White'])/df_2012['2012_Persons'] * 100

  df_2019["2019_Percentage_College"] = df_2019['2019_Bachelor_Degree']/df_2019['2019_Persons'] * 100
  df_2019["2019_Percentage_Non_White"] = (df_2019['2019_Persons'] - df_2019['2019_Count_White'])/df_2019['2019_Persons'] * 100

  df_final = pd.DataFrame()
  df_new = df_2012.merge(df_2019, on='GeoID')
  # df_new = df_new.dropna()

  df_final['GeoID'] = df_new['GeoID']
  df_final["Percent_Change_Income"] = (df_new['2019_Median_Income']-df_new['2012_Median_Income'])/df_new['2012_Median_Income'] * 100
  df_final["Percent_Change_Home"] = (df_new['2019_Median_Home_Value']-df_new['2012_Median_Home_Value'])/df_new['2012_Median_Home_Value'] * 100
  df_final["Percent_Change_Education"] = (df_new['2019_Percentage_College']-df_new['2012_Percentage_College'])/df_new['2012_Percentage_College'] * 100
  df_final["Percent_Change_Race"] = (df_new['2019_Percentage_Non_White']-df_new['2012_Percentage_Non_White'])/df_new['2012_Percentage_Non_White'] * 100

  if income_percentile_40 is None and home_value_percentile_40 is None:
    income_percentile_40 = df_new['2012_Median_Income'].quantile(0.4)
    home_value_percentile_40 = df_new['2012_Median_Home_Value'].quantile(0.4)

  df_final['eligible'] = ((df_new['2012_Median_Income'] <= income_percentile_40) & (df_new['2012_Median_Home_Value'] <= home_value_percentile_40)).astype(int)

  if education_threshold is None and home_threshold is None:
    education_threshold = df_final['Percent_Change_Education'].quantile(2/3)
    home_threshold = df_final['Percent_Change_Home'].quantile(2/3)

  df_final['label'] = ((df_final['Percent_Change_Education'] > education_threshold) & (df_final['Percent_Change_Home'] > home_threshold) & df_final['eligible'] == 1).astype(int)
  return df_final, income_percentile_40, home_value_percentile_40, education_threshold, home_threshold

### Census tract extraction

In [10]:
# Davidson County
county = 'geoId/47037'

In [11]:
# Get lists of census tracts within the County, respectively.
census_tracts = dc.get_places_in([county], 'CensusTract')[county]

In [12]:
stats = dc.get_stat_all(census_tracts, ["Median_Income_Household","Median_HomeValue_HousingUnit_OccupiedHousingUnit_OwnerOccupied", "Count_Person_EducationalAttainmentBachelorsDegreeOrHigher", 'Count_Person_WhiteAlone',"Count_Person"])

In [13]:
geo_ids = list(stats.keys())
len(geo_ids)

161

In [14]:
# Create DataFrames for each year
df_2012 = process_data_for_year('2012', stats)
df_2019 = process_data_for_year('2019', stats)

# Get Features
df_census_tracts, ip4,hp4,ep,hp = get_features_census_tracts(df_2012,df_2019)
df_census_tracts

,GeoID,Percent_Change_Income,Percent_Change_Home,Percent_Change_Education,Percent_Change_Race,eligible,label
0,geoId/47037010702,10.403512,40.737564,40.081976,19.679303,1,0
1,geoId/47037011700,115.567794,124.535554,114.648886,-11.483863,0,0
2,geoId/47037013201,18.137818,59.118644,12.325231,-28.251171,0,0
3,geoId/47037019003,46.901352,10.564784,97.523760,-11.089869,0,0
4,geoId/47037019114,15.193223,27.232143,-5.772429,11.899245,0,0
...,...,...,...,...,...,...,...
152,geoId/47037011200,33.476369,78.235673,45.993140,-22.123051,0,0
153,geoId/47037015623,1.088139,20.379747,11.109573,-39.465802,0,0
154,geoId/47037017701,49.464953,84.306060,4.012711,11.881930,0,0
155,geoId/47037018404,46.612046,16.240071,2.303056,211.543622,0,0


### Census block group extraction

In [15]:
count = 0
block_groups = {}
for ct in census_tracts:
  blocks = dc.get_places_in([ct], 'CensusBlockGroup')[ct]
  stats = dc.get_stat_all(blocks, ["Median_Income_Household","Median_HomeValue_HousingUnit_OccupiedHousingUnit_OwnerOccupied", "Count_Person_EducationalAttainmentBachelorsDegreeOrHigher", 'Count_Person_WhiteAlone',"Count_Person"])
  df_2012 = process_data_for_year('2012', stats)
  df_2019 = process_data_for_year('2019', stats)
  try:
    df_final, _,_,_,_ = get_features_census_tracts(df_2012,df_2019,ip4,hp4,ep,hp)
    block_groups[ct] = df_final
  except:
    print("block group data empty for ct: ", ct)

block group data empty for ct:  geoId/47037013000
block group data empty for ct:  geoId/47037013602
block group data empty for ct:  geoId/47037014400
block group data empty for ct:  geoId/47037014800
block group data empty for ct:  geoId/47037016500
block group data empty for ct:  geoId/47037980100
block group data empty for ct:  geoId/47037980200


In [16]:
# with open('block_groups2.pkl', 'wb') as file:
#     pickle.dump(block_groups, file)

## Data prep for Training and Testing

In [17]:
# Load in edge attributes
with open('/edge_attr.pkl', 'rb') as file:
  edge_attr = pickle.load(file)

data_list = []
graph_labels_list = []

for id in block_groups.keys():
  features = block_groups[id][['Percent_Change_Income', 'Percent_Change_Home', 'Percent_Change_Education', 'Percent_Change_Race']].to_numpy()

  node_features = torch.tensor(features, dtype=torch.float)
  neighbors = edge_attr[id]

  if neighbors and max(max(neighbors)) < len(features):
    edge_index = torch.tensor(neighbors, dtype=torch.long).t().contiguous()

    # Node-level labels and a graph-level label
    node_labels = torch.tensor(block_groups[id]['label'].values)  # binary labels for node-level task
    graph_label = torch.tensor([df_census_tracts[df_census_tracts['GeoID'] == id]['label'].iloc[0]], dtype=torch.long)

    # PyTorch Geometric Data object
    data = Data(x=node_features, edge_index=edge_index, y=node_labels)
    data.y_graph = graph_label  # Add graph-level label as an additional attribute
    data_list.append(data)

cleaned_data_list = []
for data in data_list:
  if torch.isnan(data.x).any() or torch.isnan(data.y).any() or torch.isinf(data.x).any() or torch.isinf(data.y).any():
    continue  # Skip this Data object
  cleaned_data_list.append(data)

# Data object has a y_graph attribute that represents the graph-level label
graph_labels = [data.y_graph.item() for data in cleaned_data_list]

In [18]:
len(data_list)

131

In [19]:
len(cleaned_data_list)

75

### Training Test Split

In [20]:
# train_data, test_data = train_test_split(cleaned_data_list, test_size=0.1, stratify=graph_labels)

# Create a DataLoader to batch Data objects
# train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
# test_loader = DataLoader(test_data,batch_size=16,shuffle=False)

### Create Model

In [47]:
model = MultitaskGCN(num_node_features=4, num_classes_node=2, num_classes_graph=2)

### Set Optimizers

In [48]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn_node = torch.nn.CrossEntropyLoss()  # For node-level classification
loss_fn_graph = torch.nn.CrossEntropyLoss()  # For graph-level classification

### Train

In [23]:
# train(model, optimizer, train_loader, loss_fn_node, loss_fn_graph)

### Evaulate

In [24]:
# node_acc, graph_acc = evaluate(model, test_loader)
# print(f'Node-level Accuracy: {node_acc:.4f}, Graph-level Accuracy: {graph_acc:.4f}')

### K-fold cross validation

In [53]:
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
results = []
# Perform k-fold cross-validation
for fold, (train_idx, test_idx) in enumerate(kf.split(cleaned_data_list)):
    train_data = [cleaned_data_list[i] for i in train_idx]
    test_data = [cleaned_data_list[i] for i in test_idx]

    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    for data in train_loader:
      assert not torch.isinf(data.x).any(), "NaNs in input features"
      assert not torch.isinf(data.y).any(), "NaNs in node labels"
      assert not torch.isinf(data.y_graph).any(), "NaNs in graph labels"

    test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

    # Training phase
    train(model, optimizer, train_loader, loss_fn_node, loss_fn_graph)

    # Evaluation phase
    result = evaluate(model,test_loader)
    results.append((result['Node Accuracy'], result['Graph Accuracy']))
    print(f"Fold {fold + 1}/{num_folds}")
    # print(f"Training Loss: {train_loss:.4f}")
    print(f"Node-level Accuracy: {result['Node Accuracy']:.4f}, Graph-level Accuracy: {result['Graph Accuracy']:.4f}")

# Aggregate and display final results
average_node_accuracy = sum([res[0] for res in results]) / num_folds
average_graph_accuracy = sum([res[1] for res in results]) / num_folds
print(f"Average Node-level Accuracy: {average_node_accuracy:.4f}")
print(f"Average Graph-level Accuracy: {average_graph_accuracy:.4f}")


Epoch 1, Average Loss: 0.0176
Epoch 2, Average Loss: 0.0196
Epoch 3, Average Loss: 0.0193
Epoch 4, Average Loss: 0.0192
Epoch 5, Average Loss: 0.0189
Epoch 6, Average Loss: 0.0157
Epoch 7, Average Loss: 0.0190
Epoch 8, Average Loss: 0.0166
Epoch 9, Average Loss: 0.0202
Epoch 10, Average Loss: 0.0156
Epoch 11, Average Loss: 0.0199
Epoch 12, Average Loss: 0.0244
Epoch 13, Average Loss: 0.0164
Epoch 14, Average Loss: 0.0183
Epoch 15, Average Loss: 0.0196
Epoch 16, Average Loss: 0.0198
Epoch 17, Average Loss: 0.0199
Epoch 18, Average Loss: 0.0250
Epoch 19, Average Loss: 0.0146
Epoch 20, Average Loss: 0.0197
Epoch 21, Average Loss: 0.0169
Epoch 22, Average Loss: 0.0173
Epoch 23, Average Loss: 0.0173
Epoch 24, Average Loss: 0.0170
Epoch 25, Average Loss: 0.0256
Epoch 26, Average Loss: 0.0173
Epoch 27, Average Loss: 0.0324
Epoch 28, Average Loss: 0.0179
Epoch 29, Average Loss: 0.0296
Epoch 30, Average Loss: 0.0244
Epoch 31, Average Loss: 5.3769
Epoch 32, Average Loss: 0.4719
Epoch 33, Average

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 9, Average Loss: 0.1841
Epoch 10, Average Loss: 0.1361
Epoch 11, Average Loss: 0.0999
Epoch 12, Average Loss: 0.0936
Epoch 13, Average Loss: 0.1051
Epoch 14, Average Loss: 0.0922
Epoch 15, Average Loss: 0.0790
Epoch 16, Average Loss: 0.0610
Epoch 17, Average Loss: 0.0604
Epoch 18, Average Loss: 0.0668
Epoch 19, Average Loss: 0.0416
Epoch 20, Average Loss: 0.0408
Epoch 21, Average Loss: 0.0353
Epoch 22, Average Loss: 0.0348
Epoch 23, Average Loss: 0.0393
Epoch 24, Average Loss: 0.0550
Epoch 25, Average Loss: 0.0465
Epoch 26, Average Loss: 0.0311
Epoch 27, Average Loss: 0.0363
Epoch 28, Average Loss: 0.0267
Epoch 29, Average Loss: 0.0451
Epoch 30, Average Loss: 0.0553
Epoch 31, Average Loss: 0.0282
Epoch 32, Average Loss: 0.0242
Epoch 33, Average Loss: 0.0310
Epoch 34, Average Loss: 0.0232
Epoch 35, Average Loss: 0.0374
Epoch 36, Average Loss: 0.0424
Epoch 37, Average Loss: 0.0218
Epoch 38, Average Loss: 0.0909
Epoch 39, Average Loss: 0.0309
Epoch 40, Average Loss: 0.0578
Epoch 41,

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 11, Average Loss: 0.3149
Epoch 12, Average Loss: 0.1349
Epoch 13, Average Loss: 0.1525
Epoch 14, Average Loss: 0.1154
Epoch 15, Average Loss: 0.1241
Epoch 16, Average Loss: 0.0841
Epoch 17, Average Loss: 0.0791
Epoch 18, Average Loss: 0.0745
Epoch 19, Average Loss: 0.0756
Epoch 20, Average Loss: 0.0672
Epoch 21, Average Loss: 0.0581
Epoch 22, Average Loss: 0.0760
Epoch 23, Average Loss: 0.0496
Epoch 24, Average Loss: 0.0604
Epoch 25, Average Loss: 0.0511
Epoch 26, Average Loss: 0.0473
Epoch 27, Average Loss: 0.0502
Epoch 28, Average Loss: 0.0423
Epoch 29, Average Loss: 0.0480
Epoch 30, Average Loss: 0.0407
Epoch 31, Average Loss: 0.0516
Epoch 32, Average Loss: 0.0453
Epoch 33, Average Loss: 0.0389
Epoch 34, Average Loss: 0.0391
Epoch 35, Average Loss: 0.0442
Epoch 36, Average Loss: 0.0408
Epoch 37, Average Loss: 0.0400
Epoch 38, Average Loss: 0.0401
Epoch 39, Average Loss: 0.0375
Epoch 40, Average Loss: 0.0462
Epoch 41, Average Loss: 0.0414
Epoch 42, Average Loss: 0.0431
Epoch 43

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 6, Average Loss: 0.7046
Epoch 7, Average Loss: 0.4246
Epoch 8, Average Loss: 0.3592
Epoch 9, Average Loss: 1.0567
Epoch 10, Average Loss: 0.1577
Epoch 11, Average Loss: 0.4014
Epoch 12, Average Loss: 0.3296
Epoch 13, Average Loss: 0.4437
Epoch 14, Average Loss: 0.3265
Epoch 15, Average Loss: 0.2041
Epoch 16, Average Loss: 0.1843
Epoch 17, Average Loss: 0.1408
Epoch 18, Average Loss: 0.2075
Epoch 19, Average Loss: 0.1708
Epoch 20, Average Loss: 0.1523
Epoch 21, Average Loss: 0.1126
Epoch 22, Average Loss: 0.1119
Epoch 23, Average Loss: 0.1068
Epoch 24, Average Loss: 0.0925
Epoch 25, Average Loss: 0.0974
Epoch 26, Average Loss: 0.0943
Epoch 27, Average Loss: 0.1033
Epoch 28, Average Loss: 0.0909
Epoch 29, Average Loss: 0.0848
Epoch 30, Average Loss: 0.1006
Epoch 31, Average Loss: 0.0874
Epoch 32, Average Loss: 0.0808
Epoch 33, Average Loss: 0.0864
Epoch 34, Average Loss: 0.0764
Epoch 35, Average Loss: 0.0825
Epoch 36, Average Loss: 0.0821
Epoch 37, Average Loss: 0.0781
Epoch 38, Av

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 2, Average Loss: 0.8455
Epoch 3, Average Loss: 1.4574
Epoch 4, Average Loss: 1.1216
Epoch 5, Average Loss: 0.3808
Epoch 6, Average Loss: 0.4602
Epoch 7, Average Loss: 0.5846
Epoch 8, Average Loss: 0.4577
Epoch 9, Average Loss: 0.4591
Epoch 10, Average Loss: 0.6223
Epoch 11, Average Loss: 0.8564
Epoch 12, Average Loss: 1.5157
Epoch 13, Average Loss: 0.3930
Epoch 14, Average Loss: 0.8085
Epoch 15, Average Loss: 0.6490
Epoch 16, Average Loss: 0.3828
Epoch 17, Average Loss: 0.3883
Epoch 18, Average Loss: 0.3465
Epoch 19, Average Loss: 0.4213
Epoch 20, Average Loss: 0.3742
Epoch 21, Average Loss: 0.3466
Epoch 22, Average Loss: 0.3325
Epoch 23, Average Loss: 0.3299
Epoch 24, Average Loss: 0.3390
Epoch 25, Average Loss: 0.3397
Epoch 26, Average Loss: 0.3099
Epoch 27, Average Loss: 0.3016
Epoch 28, Average Loss: 0.3437
Epoch 29, Average Loss: 0.3035
Epoch 30, Average Loss: 0.3097
Epoch 31, Average Loss: 0.2975
Epoch 32, Average Loss: 0.3006
Epoch 33, Average Loss: 0.2833
Epoch 34, Averag

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
